In [ ]:
%load_ext autoreload
%autoreload 2
from visualize import (
    shot_frames_navigator,
    animate_pitch,
    animate_chain,
    manual_labeling,
)

from stats_scraping import enrich_roosters
from data import load_and_process_soccer_events, load_and_process_metadata, load_and_process_roosters
from label import pos_labeling
from IPython.display import HTML

In [ ]:
event_df, players_df = load_and_process_soccer_events("/home/soccerdata/FIFA_WorldCup_2022/Event Data")
metadata_df = load_and_process_metadata("/home/soccerdata/FIFA_WorldCup_2022/Metadata")

In [ ]:
# 808 unique players, some teams have less than 26 players. 
rooster_df = load_and_process_roosters("/home/soccerdata/FIFA_WorldCup_2022/Rosters")
rooster_df

In [ ]:
enriched_roosters = enrich_roosters(rooster_df)